In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from acquire import get_connection, new_telco_churn_data, get_telco_churn_data


In [2]:
df = get_telco_churn_data()


In [3]:
df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [4]:
df.head(1)
df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check


In [6]:
df.senior_citizen.value_counts()

0    5901
1    1142
Name: senior_citizen, dtype: int64

In [ ]:
df1 = df[['payment_type_id', 'payment_type']]

In [ ]:
df1.value_counts()

Got values for payment type good to cut one


In [ ]:
df.head(1)

In [ ]:
df[['contract_type_id', 'contract_type']].value_counts()

In [ ]:
df.dtypes

In [ ]:
df.total_charges = pd.to_numeric(df.total_charges, errors='coerce').astype('float64')


In [ ]:
df[df.total_charges.isnull()]

In [ ]:
df.total_charges = df.total_charges.fillna(value=0)

In [ ]:
df[df.total_charges.isnull()]


# usable code

In [ ]:
def clean_telco_df(df):
    # Make total_charges a float instead of an object. 
    df.total_charges = pd.to_numeric(df.total_charges, errors='coerce').astype('float64')
    # Customers with 0 tenure have null values, made total_charges 0 for those customers because they have not yet paid a bill
    df.total_charges = df.total_charges.fillna(value=0)
    # Customers who do not have internet or phone service are the same as those not recieving the extra service by choice.
    df.replace('No internet service', 'No', inplace=True)
    df.replace('No phone service', 'No', inplace = True)
    # encode all non-numeric data that I can afford to drop the first column for
    dummy_df = pd.get_dummies(df[['gender',
                                  'partner',
                                  'dependents',
                                  'phone_service',
                                  'multiple_lines',
                                  'online_security', 
                                  'online_backup', 
                                  'device_protection', 
                                  'tech_support', 
                                  'streaming_tv',
                                  'streaming_movies',
                                  'paperless_billing',
                                  'churn' ]],
                              drop_first=True)
    # create readable column titles
    dummy_df = dummy_df.rename(columns={'gender_Male': 'is_male',
                                   'partner_Yes': 'has_partner',
                                   'dependents_Yes': 'has_dependents',
                                   'phone_service_Yes': 'has_phone_service',
                                   'multiple_lines_Yes': 'has_multiple_lines',
                                   'online_security_Yes': 'has_online_security',
                                   'online_backup_Yes': 'has_online_backup',
                                   'device_protection_Yes': 'has_device_protection',
                                   'tech_support_Yes': 'has_tech_support',
                                   'streaming_tv_Yes': 'has_streaming_tv',
                                   'streaming_movies_Yes': 'has_streaming_movies',
                                   'paperless_billing_Yes': 'has_paperless_billing',
                                   'churn_Yes': 'has_churned'})
    # Drop all columns that i just made an encoding for, also dropped duplicate columns 
    # payment_type_id, contract_type_id,internet_service_type_id. Dropped customer_id as it holds no value
    df = df.drop(columns =['gender',
                       'partner',
                       'dependents',
                       'phone_service',
                       'multiple_lines',
                       'online_security', 
                       'online_backup', 
                       'device_protection', 
                       'tech_support', 
                       'streaming_tv',
                       'streaming_movies',
                       'paperless_billing',
                       'churn',
                       'customer_id',
                       'payment_type_id',
                       'contract_type_id', 
                       'internet_service_type_id'])
    # merge the dummy_df and df
    df = pd.concat([df, dummy_df], axis =1)
    # Seperated payment type and whether or not a payment was automatic
    df["automatic_payment"] =( df.payment_type == ('Bank transfer (automatic)')) | (df.payment_type == ('Credit card (automatic)'))
    df["automatic_payment"] = (df["automatic_payment"]).astype(int)
    # create another df with contract types, payment types, and internet service
    dummy_df2 = pd.get_dummies(df[['internet_service_type','contract_type','payment_type']], drop_first=False )
    #rename dummy table columns
    #rename columns
    dummy_df2 = dummy_df2.rename(columns={'internet_service_type_DSL': 'has_dsl',
                                   'internet_service_type_Fiber optic': 'has_fiber',
                                   'internet_service_type_None': 'has_no_internet',
                                   'contract_type_Month-to-month': 'month_to_month_customer',
                                   'contract_type_One year': 'contract_customer_one_year',
                                   'contract_type_Two year': 'contract_customer_two_year',
                                   'payment_type_Bank transfer (automatic)': 'pays_by_bank_transfer',
                                   'payment_type_Credit card (automatic)': 'pays_by_credit_card',
                                   'payment_type_Electronic check': 'pays_by_electronic_check',
                                   'payment_type_Mailed check': 'pays_by_mailed_check'
                                   })
    #concat new columns for contract types, payment types, amd internet service
    df = pd.concat([df, dummy_df2], axis =1)
    # drop duplicate info columns
    df = df.drop(columns =['internet_service_type',
                       'contract_type',
                       'payment_type'
                       ])
    return df

In [ ]:
df= clean_telco_df(df)

In [ ]:
df.info()

In [ ]:
df[['tech_support','streaming_tv','streaming_movies']].value_counts()

In [ ]:
# Customers who do not have internet or phone service are the same as those not recieving the extra service by choice.
df.replace('No internet service', 'No', inplace=True)
df.replace('No phone service', 'No', inplace = True)

In [ ]:
df

In [ ]:
df[['tech_support','streaming_tv','streaming_movies']].value_counts()

In [8]:
# encode all non-numeric data
dummy_df = pd.get_dummies(df[['gender',
                              'partner',
                              'dependents',
                              'phone_service',
                              'multiple_lines',
                              'online_security', 
                              'online_backup', 
                              'device_protection', 
                              'tech_support', 
                              'streaming_tv',
                              'streaming_movies',
                              'paperless_billing',
                              'churn' ]],
                          drop_first=True)

In [ ]:
dummy_df.head()

In [9]:
# create readable column titles
dummy_df = dummy_df.rename(columns={'gender_Male': 'is_male',
                                   'partner_Yes': 'has_partner',
                                   'dependents_Yes': 'has_dependents',
                                   'phone_service_Yes': 'has_phone_service',
                                   'multiple_lines_Yes': 'has_multiple_lines',
                                   'online_security_Yes': 'has_online_security',
                                   'online_backup_Yes': 'has_online_backup',
                                   'device_protection_Yes': 'has_device_protection',
                                   'tech_support_Yes': 'has_tech_support',
                                   'streaming_tv_Yes': 'has_streaming_tv',
                                   'streaming_movies_Yes': 'has_streaming_movies',
                                   'paperless_billing_Yes': 'has_paperless_billing',
                                   'churn_Yes': 'has_churned'})

In [ ]:
dummy_df.head()

In [ ]:
df.head()

In [ ]:
# Drop all columns that i just made an encoding for, also dropped duplicate columns 
# payment_type_id, contract_type_id,internet_service_type_id. Dropped customer_id as it holds no value
df = df.drop(columns =['gender',
                       'partner',
                       'dependents',
                       'phone_service',
                       'multiple_lines',
                       'online_security', 
                       'online_backup', 
                       'device_protection', 
                       'tech_support', 
                       'streaming_tv',
                       'streaming_movies',
                       'paperless_billing',
                       'churn',
                       'customer_id',
                       'payment_type_id',
                       'contract_type_id', 
                       'internet_service_type_id'
                      ])

In [ ]:
df.head()

In [10]:
# merge the dummy_df and df
df = pd.concat([df, dummy_df], axis =1)

In [19]:
df[["gender", "is_male"]]

,gender,is_male
0,Female,0
1,Female,0
2,Male,1
3,Male,1
4,Female,0
...,...,...
7038,Male,1
7039,Female,0
7040,Female,0
7041,Male,1


In [ ]:
dummy_df2 = pd.get_dummies(df[['internet_service_type','contract_type','payment_type']], drop_first=False )

In [ ]:
dummy_df2.head()


In [ ]:
#rename columns
dummy_df2 = dummy_df2.rename(columns={'internet_service_type_DSL': 'has_dsl',
                                   'internet_service_type_Fiber optic': 'has_fiber',
                                   'internet_service_type_None': 'has_no_internet',
                                   'contract_type_Month-to-month': 'month_to_month_customer',
                                   'contract_type_One year': 'contract_customer_one_year',
                                   'contract_type_Two year': 'contract_customer_two_year',
                                   'payment_type_Bank transfer (automatic)': 'pays_by_bank_transfer',
                                   'payment_type_Credit card (automatic)': 'pays_by_credit_card',
                                   'payment_type_Electronic check': 'pays_by_electronic_check',
                                   'payment_type_Mailed check': 'pays_by_mailed_check'
                                   })

In [ ]:
dummy_df2

In [ ]:
df.payment_type.value_counts()

In [ ]:
# Seperated payment type and whether or not a payment was automatic
df["automatic_payment"] =( df.payment_type == ('Bank transfer (automatic)')) | (df.payment_type == ('Credit card (automatic)'))
df["automatic_payment"] = (df["automatic_payment"]).astype(int)

In [ ]:
df

In [ ]:
df = pd.concat([df, dummy_df2], axis =1)

In [ ]:
df

In [ ]:
df = df.drop(columns =['internet_service_type',
                       'contract_type',
                       'payment_type'
                       ])

In [ ]:
df

In [ ]:
def telco_split(df):
    #splitting our data
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.has_churned)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.has_churned)
    return train, validate, test


In [ ]:
telco_churn_split(df)

In [ ]:
def prep_telco(df):
    #cleaning and splitting our data
    df = clean_telco_df(df)
    train, validate, test = telco_split(df)
    return train, validate, test

In [ ]:
prep_telco_churn(df)

In [ ]:
train

In [ ]:

#prep function will split this clean data into train, validate, and test sets.
train, validate, test = prep_telco(df)

In [ ]:
train.shape, validate.shape , test.shape

In [ ]:
train
